Going to start with cleaning the weather airport data first, because it's simpler than the energy data. 

In [172]:
from gcp_interaction import read_blob_to_pandas, write_to_blob_csv
import json
import pandas as pd

In [158]:
with open("excess-energy-prediction-393ec78547e4.json", "r") as f:
    gcp_login_info = json.load(f)
with open("gcp_info.json", "r") as f:
    bucket_name = json.load(f)["bucket_name"]

In [159]:
def clean_weather_df(df: pd.DataFrame) -> pd.DataFrame:
    MIN_DATE = "2017-01-01 00:00:00"
    df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True, errors="ignore")
    # fix space value
    df["wdsp"] = df["wdsp"].replace(to_replace=" ", value=0)
    df["wdsp"] = df["wdsp"].astype("int32")
    # Reduce timeframe
    df = df[(df["date"] >= MIN_DATE)]
    return df

# Belmullet

In [160]:
belmullet_dir = r"Raw_Data/met/BELMULLET/belmullet.csv"
belmullet_df = read_blob_to_pandas(
    bucket_name, belmullet_dir, header=20, low_memory=False
)
belmullet_df = clean_weather_df(belmullet_df)

In [161]:
belmullet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53329 entries, 528513 to 581841
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    53329 non-null  datetime64[ns]
 1   ind     53329 non-null  int64         
 2   rain    53329 non-null  float64       
 3   ind.1   53329 non-null  int64         
 4   temp    53329 non-null  float64       
 5   ind.2   53329 non-null  int64         
 6   wetb    53329 non-null  float64       
 7   dewpt   53329 non-null  float64       
 8   vappr   53329 non-null  float64       
 9   rhum    53329 non-null  int64         
 10  msl     53329 non-null  float64       
 11  ind.3   53329 non-null  int64         
 12  wdsp    53329 non-null  int32         
 13  ind.4   53329 non-null  int64         
 14  wddir   53329 non-null  object        
 15  ww      53329 non-null  object        
 16  w       53329 non-null  object        
 17  sun     53329 non-null  object        
 18  

In [162]:
belmullet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53329 entries, 528513 to 581841
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    53329 non-null  datetime64[ns]
 1   ind     53329 non-null  int64         
 2   rain    53329 non-null  float64       
 3   ind.1   53329 non-null  int64         
 4   temp    53329 non-null  float64       
 5   ind.2   53329 non-null  int64         
 6   wetb    53329 non-null  float64       
 7   dewpt   53329 non-null  float64       
 8   vappr   53329 non-null  float64       
 9   rhum    53329 non-null  int64         
 10  msl     53329 non-null  float64       
 11  ind.3   53329 non-null  int64         
 12  wdsp    53329 non-null  int32         
 13  ind.4   53329 non-null  int64         
 14  wddir   53329 non-null  object        
 15  ww      53329 non-null  object        
 16  w       53329 non-null  object        
 17  sun     53329 non-null  object        
 18  

# Dublin

In [163]:
dublin_dir = r"Raw_Data/met/DUBLIN AIRPORT/dublin.csv"
dublin_df = read_blob_to_pandas(bucket_name, dublin_dir, header=20, low_memory=False)
dublin_df = clean_weather_df(dublin_df)

In [164]:
dublin_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53240 entries, 648696 to 701935
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    53240 non-null  datetime64[ns]
 1   ind     53240 non-null  int64         
 2   rain    53240 non-null  float64       
 3   ind.1   53240 non-null  int64         
 4   temp    53240 non-null  float64       
 5   ind.2   53240 non-null  int64         
 6   wetb    53240 non-null  float64       
 7   dewpt   53240 non-null  float64       
 8   vappr   53240 non-null  object        
 9   rhum    53240 non-null  object        
 10  msl     53240 non-null  float64       
 11  ind.3   53240 non-null  int64         
 12  wdsp    53240 non-null  int32         
 13  ind.4   53240 non-null  int64         
 14  wddir   53240 non-null  object        
 15  ww      53240 non-null  int64         
 16  w       53240 non-null  int64         
 17  sun     53240 non-null  float64       
 18  

# Shannon

In [165]:
shannon_dir = r"Raw_Data/met/SHANNON AIRPORT/shannon.csv"
shannon_df = read_blob_to_pandas(bucket_name, shannon_dir, header=20, low_memory=False)
shannon_df = clean_weather_df(shannon_df)

In [166]:
shannon_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53240 entries, 625319 to 678558
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    53240 non-null  datetime64[ns]
 1   ind     53240 non-null  int64         
 2   rain    53240 non-null  float64       
 3   ind.1   53240 non-null  int64         
 4   temp    53240 non-null  float64       
 5   ind.2   53240 non-null  int64         
 6   wetb    53240 non-null  object        
 7   dewpt   53240 non-null  float64       
 8   vappr   53240 non-null  float64       
 9   rhum    53240 non-null  int64         
 10  msl     53240 non-null  float64       
 11  ind.3   53240 non-null  int64         
 12  wdsp    53240 non-null  int32         
 13  ind.4   53240 non-null  int64         
 14  wddir   53240 non-null  object        
 15  ww      53240 non-null  object        
 16  w       53240 non-null  object        
 17  sun     53240 non-null  float64       
 18  

# Cork

In [167]:
cork_dir = r"Raw_Data/met/CORK AIRPORT/cork.csv"
cork_df = read_blob_to_pandas(bucket_name, cork_dir, header=20, low_memory=False)
cork_df = clean_weather_df(cork_df)

In [168]:
cork_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53240 entries, 482135 to 535374
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    53240 non-null  datetime64[ns]
 1   ind     53240 non-null  int64         
 2   rain    53240 non-null  object        
 3   ind.1   53240 non-null  int64         
 4   temp    53240 non-null  float64       
 5   ind.2   53240 non-null  int64         
 6   wetb    53240 non-null  object        
 7   dewpt   53240 non-null  float64       
 8   vappr   53240 non-null  object        
 9   rhum    53240 non-null  object        
 10  msl     53240 non-null  float64       
 11  ind.3   53240 non-null  int64         
 12  wdsp    53240 non-null  int32         
 13  ind.4   53240 non-null  int64         
 14  wddir   53240 non-null  int64         
 15  ww      53240 non-null  int64         
 16  w       53240 non-null  int64         
 17  sun     53240 non-null  object        
 18  

# Combining

In [169]:
merged_df = pd.merge(belmullet_df, dublin_df, on="date", suffixes=("_BEL", "_DUB"))

merged_df = pd.merge(
    merged_df,
    shannon_df,
    on="date",
)
renamed_cols = [x + "_SHAN" if x != "date" else x for x in shannon_df.columns]
merged_df = merged_df.rename(columns=dict(zip(shannon_df.columns, renamed_cols)))

merged_df = pd.merge(
    merged_df,
    cork_df,
    on="date",
)
renamed_cols = [x + "_CORK" if x != "date" else x for x in cork_df.columns]
merged_df = merged_df.rename(columns=dict(zip(cork_df.columns, renamed_cols)))

In [170]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53240 entries, 0 to 53239
Data columns (total 81 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        53240 non-null  datetime64[ns]
 1   ind_BEL     53240 non-null  int64         
 2   rain_BEL    53240 non-null  float64       
 3   ind.1_BEL   53240 non-null  int64         
 4   temp_BEL    53240 non-null  float64       
 5   ind.2_BEL   53240 non-null  int64         
 6   wetb_BEL    53240 non-null  float64       
 7   dewpt_BEL   53240 non-null  float64       
 8   vappr_BEL   53240 non-null  float64       
 9   rhum_BEL    53240 non-null  int64         
 10  msl_BEL     53240 non-null  float64       
 11  ind.3_BEL   53240 non-null  int64         
 12  wdsp_BEL    53240 non-null  int32         
 13  ind.4_BEL   53240 non-null  int64         
 14  wddir_BEL   53240 non-null  object        
 15  ww_BEL      53240 non-null  object        
 16  w_BEL       53240 non-

In [171]:
merged_df.head()

,date,ind_BEL,rain_BEL,ind.1_BEL,temp_BEL,ind.2_BEL,wetb_BEL,dewpt_BEL,vappr_BEL,rhum_BEL,...,ind.3_CORK,wdsp_CORK,ind.4_CORK,wddir_CORK,ww_CORK,w_CORK,sun_CORK,vis_CORK,clht_CORK,clamt_CORK
0,2017-01-01 00:00:00,0,0.0,0,5.2,0,3.9,2.0,7.1,79,...,2,11,2,230,61,62,0.0,9000,22,8
1,2017-01-01 01:00:00,0,0.5,0,4.7,0,3.4,1.4,6.8,78,...,2,16,2,320,61,66,0.0,20000,8,8
2,2017-01-01 02:00:00,0,0.0,0,5.7,0,3.8,0.7,6.4,70,...,2,16,2,350,21,62,0.0,25000,80,7
3,2017-01-01 03:00:00,0,0.4,0,5.6,0,3.3,-0.6,5.9,64,...,2,14,2,340,2,62,0.0,25000,10,7
4,2017-01-01 04:00:00,0,0.6,0,4.7,0,3.1,0.5,6.3,74,...,2,14,2,350,21,62,0.0,30000,50,7


# Writing back to GCP

reading in all these CSVs is slow going, so I'm writing back to GCP so I can import a single df and clean in another file. 

In [173]:
write_to_blob_csv(
    bucket_name=bucket_name, df=merged_df, filepath="transformed/weather_combined.csv"
)

transformed/weather_combined.csv uploaded to <Bucket: excess-energy-raw-data>
